<a href="https://colab.research.google.com/github/lamaachi/MyDataScience_Projects/blob/main/RNN_Reuters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from tensorflow.keras.datasets import reuters
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense,Dropout
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping


In [30]:
num_words = 10000  # Vocabulaire limité
maxlen = 500          # Longueur maximale des séquences
num_classes = 46      # Nombre de classes

In [31]:
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=num_words,
                                                         maxlen=maxlen)
x_train = pad_sequences(x_train, maxlen=200)
x_test = pad_sequences(x_test, maxlen=200)


In [32]:
print("Nombre de séquences d'entraînement:", len(x_train))
print("Nombre de séquences de test:", len(x_test))
print("Nombre de classes:", num_classes)
print("Exemple de séquence d'entraînement (indices de mots):", x_train[0])
print("Étiquette correspondante:", y_train[0])

Nombre de séquences d'entraînement: 8621
Nombre de séquences de test: 2156
Nombre de classes: 46
Exemple de séquence d'entraînement (indices de mots): [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    1    2    2    8   43   10  447    5   25  207  270    5 3095
  111   16  369  186   90   67    7   89    5   19  102    6   19  124
   15   90   67   84   22  482   26    7   48    4   49    8  864   39
  209  154    6  151    6   83   11   15   22  155   11   15    7   

In [33]:
# -------------------------------------------------------------------------------
# 3. Construction du Modèle RNN (SimpleRNN)
# -------------------------------------------------------------------------------
model = Sequential()
embedding_dim = 128
rnn_units = 128
model.add(Embedding(num_words, embedding_dim))
model.add(SimpleRNN(rnn_units))
# La couche Dense de sortie doit avoir `num_classes` unités et une fonction
# d'activation 'softmax' pour la classification multi-classe.
model.add(Dense(num_classes, activation='softmax'))

# Afficher un résumé de l'architecture du modèle.
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_16 (SimpleRNN)            │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [41]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [42]:
# Add early stopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

In [43]:
history = model.fit(
    x_train, y_train,
    epochs=8,
    batch_size=21,
    validation_split=0.2,
    callbacks=[early_stopping]
)


Epoch 1/8
329/329 ━━━━━━━━━━━━━━━━━━━━ 12s 29ms/step - accuracy: 0.7661 - loss: 0.9370 - val_accuracy: 0.4557 - val_loss: 2.4030
Epoch 2/8
329/329 ━━━━━━━━━━━━━━━━━━━━ 18s 25ms/step - accuracy: 0.8271 - loss: 0.6776 - val_accuracy: 0.4643 - val_loss: 2.4697
Epoch 3/8
329/329 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.9098 - loss: 0.4065 - val_accuracy: 0.4632 - val_loss: 2.5510
Epoch 4/8
329/329 ━━━━━━━━━━━━━━━━━━━━ 10s 23ms/step - accuracy: 0.9421 - loss: 0.2589 - val_accuracy: 0.4400 - val_loss: 2.7908


In [37]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_acc}")


68/68 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.4964 - loss: 1.9906
Test Loss: 2.0361340045928955
Test Accuracy: 0.48005566000938416


In [ ]:
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Courbe de perte')
plt.show()


In [ ]:
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Courbe de précision')
plt.show()


In [ ]:
predictions = model.predict(x_test)
print(f"Prédiction pour un exemple : {predictions[0]}")


In [ ]:
predicted_class = predictions.argmax(axis=-1)
print(f"Classe prédite : {predicted_class[0]}")
